In [4]:
# Imports
import numpy as np
import cv2
from PIL import Image
from cv2 import resize

from sklearn import preprocessing
import skimage
from instaFilters import gotham, gingham, clarendon, juno, lark, reyes

from vgg16_places_365 import VGG16_Places365

# return scene probability vector
# suppose we have x supported scenes
NUM_SCENES = 365 # Thanks, Keras Places365
FEATURE_LENGTH = 75
def scene_vector(model, image):
    image = np.array(image, dtype=np.uint8)
    image = resize(image, (224, 224))
    image = np.expand_dims(image, 0)
    preds = model.predict(image)[0]
#     print('og shape', preds.shape)
    return preds
#     print(preds)
#     top_preds = np.argsort(preds)[::-1]
#     # load the class label
#     file_name = 'categories_places365.txt'
#     classes = list()
#     with open(file_name) as class_file:
#         for line in class_file:
#             classes.append(line.strip().split(' ')[0][3:])
#     classes = tuple(classes)
#     scene_prob = preds
#     print('--SCENE CATEGORIES:')
    # output the prediction
#     for i in range(0, 5):
#         print(classes[top_preds[i]])
#     scene_prob = np.zeros(NUM_SCENES)
#     scene_prob[0] = 1
#     return scene_prob[..., np.newaxis] # NUM_SCENES x 1

from skimage.color import rgb2hsv
# normalized feature vector
# expects float32 image
def get_features(image):
    # just a color histogram for now (remember to test hsv)
    # https://docs.opencv.org/3.1.0/d1/db7/tutorial_py_histogram_begins.html
    # remember mask param
#     image = rgb2hsv(image).astype('float32')
    # Do we normalize per histogram, or overall? (overall for performance for now)
    hist_1 = cv2.calcHist([image], [0], None, [25], [0, 1])
    hist_2 = cv2.calcHist([image], [1], None, [25], [0, 1])
    hist_3 = cv2.calcHist([image], [2], None, [25], [0, 1])
    return preprocessing.normalize(np.concatenate((hist_1, hist_2, hist_3)), axis=0)

# return dictionary of 'avg' histogram/feature vector of scenes given images (key = scenes)
# this should probably be some NN
def scene_features(model, images):
    scene_features = np.zeros((NUM_SCENES, FEATURE_LENGTH)) 
    for image in images:
        scene = scene_vector(model, image)[..., np.newaxis]
        feature = get_features(image.astype('float32'))
        img_hist = np.matmul(scene, feature.T) # image's contributions to scene features - ith row is ith scene's hist
        scene_features += img_hist
        # matrix multiply scene and feature
    # Todo: normalize each set of interesting info
    return preprocessing.normalize(scene_features)

# single version
def scene_feature(model, image):
    scene = scene_vector(model, image)[..., np.newaxis]
    feature = get_features(image.astype('float32'))
    img_hist = np.matmul(scene, feature.T)
    return img_hist
#     return preprocessing.normalize(scene_features)

Using TensorFlow backend.


In [31]:
# return array of distribution vectors for each filter fn in FILTERS
# Todo - add gotham back
FILTERS = [gotham, gingham, clarendon, juno, lark, reyes]
# Because places365 has a different set of scenes, we
# SCENES = ['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
CORPUS_CHARACTERISTIC = './scene_characteristic.npy'
DATA_DIR = 'data/images/val/'
def create_scene_characteristics():
    import glob
    model = VGG16_Places365(weights='places')
    filelist = glob.glob('{}/*.jpg'.format(DATA_DIR))[:200]
    feature_distributions = np.zeros((len(FILTERS) + 1, NUM_SCENES, FEATURE_LENGTH))
    for fname in filelist: # too scared to hold all the images in memory
        image = skimage.img_as_float(cv2.imread(fname)).astype('float32')
        feature_distributions[0] += scene_feature(model, image)
        for i, filt in enumerate(FILTERS):
            feature_distributions[i + 1] += scene_feature(model, filt(image))
    # write characteristics to file
    all_characteristics = np.asarray(feature_distributions)
    for i, filter_dist in enumerate(feature_distributions):
        feature_distributions[i] = preprocessing.normalize(filter_dist)
    np.save(CORPUS_CHARACTERISTIC, all_characteristics)
    # shape is (filters + 1) * (scenes) * (feature length)
def load_scene_characteristics():
    return np.load(CORPUS_CHARACTERISTIC)

# create_scene_characteristics()

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 

In [49]:
# Ported from stack overflow
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def classifyFilter(model, img, characteristics):
    scene_vec = scene_vector(model, img)
    # Take dominant scene for now and reference just that
    # representative scene features take into account all aspects of the scene vec
    # 1 max arg
#     scene = np.argmax(scene_vec)
    # scene_filter_features = characteristics[:, scene]
    # entire vector alternative
    characteristics = characteristics.swapaxes(0, 1) # swap scene with filter
    characteristics = characteristics.reshape((NUM_SCENES, -1)) # flatten filter-feature 
    scene_filter_features = np.matmul(scene_vec.T, characteristics)
    scene_filter_features = scene_filter_features.reshape((len(FILTERS) + 1, FEATURE_LENGTH))
    
    
    img_features = preprocessing.normalize(get_features(skimage.img_as_float(img).astype('float32')), axis=0).T
    error_vec = scene_filter_features - img_features
#     print(scene_filter_features.shape)
#     print(img_features.T.shape)
#     print(error_vec.shape)
    errors = np.linalg.norm(error_vec, axis=1)
#     print(errors)
    scores = 1 / errors
#     print(scores)
    return softmax(scores)
    
def generate_test_images(fn):
    image = cv2.imread(fn)
    fn_base = fn.split('.')[0]
    import imageio
    name = "rand"
    imageio.imwrite('data/tests/{}_base.jpg'.format(name), image)
#     print(image.shape)
    image = skimage.img_as_float(image).astype('float32')
    from skimage import img_as_ubyte
    for i, filt in enumerate(FILTERS):
        imageio.imwrite('data/tests/{}_{}.jpg'.format(name, i), img_as_ubyte(filt(image)))
generate_test_images('data/images/test/00010000.jpg')
    
def driver():
    characteristics = load_scene_characteristics()
    img = cv2.imread('data/tests/rand_0.jpg')
    model = VGG16_Places365(weights='places')
    x = classifyFilter(model, img, characteristics)    
    print('test_filter_none')
    print(np.argsort(x)[::-1])
    for filt_index in range(len(FILTERS)):
        img = cv2.imread('data/tests/rand_{}.jpg'.format(filt_index))
        x = classifyFilter(model, img, characteristics)
        print('test_filter_{}'.format(filt_index + 1))
        print(x)
        print(np.argsort(x)[::-1])


(128, 128, 3)
(128, 128, 3)


/home/joel/anaconda2/envs/pyEnv/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [50]:
driver()

test_filter_none
[0 4 3 5 1 2 6]
test_filter_1
[ 0.16873809  0.13722801  0.12436595  0.14974418  0.15388096  0.14973791
  0.11630489]
[0 4 3 5 1 2 6]
test_filter_2
[ 0.13203975  0.08514644  0.38440342  0.10652417  0.08753477  0.09793389
  0.10641755]
[2 0 3 6 5 4 1]
test_filter_3
[ 0.20210842  0.1143053   0.12210205  0.15572881  0.15114222  0.15287654
  0.10173667]
[0 3 5 4 2 1 6]
test_filter_4
[ 0.16449293  0.11481851  0.09769497  0.13552546  0.21504187  0.18376446
  0.0886618 ]
[4 5 0 3 1 2 6]
test_filter_5
[ 0.17313483  0.09934308  0.09464425  0.13297826  0.21423755  0.2024806
  0.08318143]
[4 5 0 3 1 2 6]
test_filter_6
[ 0.11894674  0.10098418  0.1575536   0.1174764   0.11250379  0.11978885
  0.27274646]
[6 2 5 0 3 4 1]
